In [13]:
import pandas as pd
import pickle 
import os
import finrl
from finenv.env_stocktrading import StockTradingEnv
from finenv.preprocessors import data_split
from finenv.save_model import *
import psutil
import ray
from datetime import datetime
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total
from ray.tune.registry import register_env
from gymnasium.wrappers import EnvCompatibility
from ray.rllib.agents import ppo
from ray.rllib.algorithms.td3 import TD3Config

#os.environ["OMP_NUM_THREADS"] = "1"
#os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

In [17]:
csv = False
if csv:
    train = pd.read_csv('dataset/train_data.csv')
    train = train.set_index(train.columns[0])
    train.index.names = ['']
    INDICATORS = ['macd','boll_ub','boll_lb','rsi_30','cci_30','dx_30','close_30_sma','close_60_sma']

else: 
    train = pd.read_pickle('dataset/qqq_train.pkl')
    train['date'] = pd.to_datetime(train['date'])
    train['date'] = train['date'].dt.strftime('%Y-%m-%d %H:%M')
    INDICATORS = ['macd','rsi','cci','adx']
    
   
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
train.head(5) 

Stock Dimension: 1, State Space: 7


,date,open,high,low,close,volume,tic,macd,rsi,cci,adx
0,2016-01-04 09:30,110.26,110.26,110.22,110.24,12700.0,QQQ,0.0,0.0,0.0,0.0
1,2016-01-04 09:45,110.14,110.14,110.00,110.00,200.0,QQQ,0.0,0.0,0.0,0.0
2,2016-01-04 10:00,110.07,110.07,110.07,110.07,400.0,QQQ,0.0,0.0,0.0,0.0
3,2016-01-04 10:15,110.05,110.05,110.02,110.03,700.0,QQQ,0.0,0.0,0.0,0.0
4,2016-01-04 10:30,109.95,110.03,109.95,110.03,800.0,QQQ,0.0,0.0,0.0,0.0


In [2]:
def env_creator(env_config):
    # env_config is passed as {} and defaults are set here
    df = env_config.get('df', train)
    hmax = env_config.get('hmax', 200)
    initial_amount = env_config.get('initial_amount', 1000000)
    num_stock_shares = env_config.get('num_stock_shares', [0] * stock_dimension)
    buy_cost_pct = env_config.get('buy_cost_pct', buy_cost_list)
    sell_cost_pct = env_config.get('sell_cost_pct', sell_cost_list)
    state_space = env_config.get('state_space', 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension)
    stock_dim = env_config.get('stock_dim', stock_dimension)
    tech_indicator_list = env_config.get('tech_indicator_list', INDICATORS)
    action_space = env_config.get('action_space', stock_dimension)
    reward_scaling = env_config.get('reward_scaling', 0.999)

    return EnvCompatibility(StockTradingEnv(
        df=df,
        hmax=hmax,
        initial_amount=initial_amount,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=buy_cost_pct,
        sell_cost_pct=sell_cost_pct,
        state_space=state_space,
        stock_dim=stock_dim,
        tech_indicator_list=tech_indicator_list,
        action_space=action_space,
        reward_scaling=reward_scaling
    ))

register_env("finrl", env_creator)

In [21]:
ray.shutdown()
#ray.init(num_cpus=122,dashboard_port=8080)
print(f"ray is being initialized")

ray is being initialized


In [4]:
from ray.rllib.algorithms.ddpg.ddpg import DDPGConfig
#config = DDPGConfig().training(lr=0.01).resources(num_gpus=1).framework(framework="torch").rollouts(num_rollout_workers=10)
config = TD3Config().training(lr=0.01).resources(num_gpus=1).framework(framework="torch").rollouts(num_rollout_workers=10)
config["model"]["fcnet_hiddens"] = [256, 256, 256]
config['train_batch_size'] = 1024
print(config.to_dict())  
# Build a Trainer object from the config and run one training iteration.
trainer = config.build(env="finrl") 

{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 1, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'num_trainer_workers': 0, 'num_gpus_per_trainer_worker': 0, 'num_cpus_per_trainer_worker': 1, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': None, 'env_config': {}, 'observation_space': None, 'action_space': None, 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'disable_env_checking': False, 'is_atari': None, 'auto_wrap_old_gym_envs': True, 'num_envs_per_worker': 1, 'samp

2023-04-07 22:24:01,127	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(RolloutWorker pid=18924) 2023-04-07 22:24:13,073	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=18924) 2023-04-07 22:24:13,073	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
(RolloutWorker pid=18924) /home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting input x to numpy array.
(RolloutWorker pid=18924)   logger.warn("Casting input x to numpy array.")
(RolloutWorker pid=18951) /home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN

In [7]:
from tqdm.notebook import tqdm
# Train away -------------------------------------------------------------
total_episodes = 200
agent_name = 'td3'
ep = 0
results = []
bar = tqdm(total=total_episodes, desc="Episodes")
date = datetime.now().strftime('%y%m%d')

while ep <= total_episodes:
    results.append(trainer.train())
    ep += 1
    bar.update(n=1)
    rwd = results[-1]['episode_reward_mean']
    if ep % 20 == 0:
        print(f'Mean Rwd:{rwd}')
    if ep % 100 == 0:
        #cwd_checkpoint = "results/checkpoints/" +  + '_' + str(ep)
        cwd_checkpoint = f"model/{agent_name}_{date}"
        trainer.save(cwd_checkpoint)
        print(f"Checkpoint{ep} saved in directory {cwd_checkpoint}")
bar.close()

Episodes:   0%|          | 0/200 [00:00<?, ?it/s]

Mean Rwd:nan
Checkpoint5 saved in directory model/ddpg_230407
Mean Rwd:nan
Checkpoint10 saved in directory model/ddpg_230407
Mean Rwd:4345951.072325179
Checkpoint15 saved in directory model/ddpg_230407
Mean Rwd:4345951.072325179
Checkpoint20 saved in directory model/ddpg_230407
Mean Rwd:4345951.072325179
Checkpoint25 saved in directory model/ddpg_230407
Mean Rwd:4345951.072325179
Checkpoint30 saved in directory model/ddpg_230407
Mean Rwd:4345951.072325179
Checkpoint35 saved in directory model/ddpg_230407
Mean Rwd:2614410.1976177725
Checkpoint40 saved in directory model/ddpg_230407
Mean Rwd:2614410.1976177725
Checkpoint45 saved in directory model/ddpg_230407
Mean Rwd:2614410.1976177725
Checkpoint50 saved in directory model/ddpg_230407
Mean Rwd:2614410.1976177725
Checkpoint55 saved in directory model/ddpg_230407
Mean Rwd:2043918.6133481797
Checkpoint60 saved in directory model/ddpg_230407
Mean Rwd:2043918.6133481797
Checkpoint65 saved in directory model/ddpg_230407
Mean Rwd:2043918.61334

2023-04-07 22:58:30,348	ERROR actor_manager.py:496 -- Ray error, taking actor 10 out of service. The actor died unexpectedly before finishing this task.
(RolloutWorker pid=20877) /home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting input x to numpy array.
(RolloutWorker pid=20877)   logger.warn("Casting input x to numpy array.")
(raylet) [2023-04-07 22:59:01,147 E 17548 17548] (raylet) node_manager.cc:3040: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: c50d046abda4a2ab6ab4bafb37b97be86733da4e69c0b14606e92e5c, IP: 172.21.0.25) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.21.0.25`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on

Mean Rwd:2043918.6133481797
Checkpoint75 saved in directory model/ddpg_230407


(raylet) [2023-04-07 23:01:01,149 E 17548 17548] (raylet) node_manager.cc:3040: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: c50d046abda4a2ab6ab4bafb37b97be86733da4e69c0b14606e92e5c, IP: 172.21.0.25) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.21.0.25`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
2023-04-07 23:01:09,521	ERROR actor_manager.py:496 -- Ray error, taking actor 10 out of service. The actor died unexpec

...

(RolloutWorker pid=11977) /home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting input x to numpy array.
(RolloutWorker pid=11977)   logger.warn("Casting input x to numpy array.")
(raylet) [2023-04-08 01:05:12,879 E 17548 17548] (raylet) node_manager.cc:3040: 27 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: c50d046abda4a2ab6ab4bafb37b97be86733da4e69c0b14606e92e5c, IP: 172.21.0.25) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.21.0.25`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_t

Mean Rwd:1309387.371752837
Checkpoint195 saved in directory model/ddpg_230407


(raylet) [2023-04-08 01:13:14,420 E 17548 17548] (raylet) node_manager.cc:3040: 78 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: c50d046abda4a2ab6ab4bafb37b97be86733da4e69c0b14606e92e5c, IP: 172.21.0.25) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.21.0.25`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2023-04-08 01:14:14,521 E 17548 17548] (raylet) node_manager.cc:3040: 77 Workers (tasks / actors) killed due

Mean Rwd:1309387.371752837
Checkpoint200 saved in directory model/ddpg_230407


(raylet) [2023-04-08 01:22:17,281 E 17548 17548] (raylet) node_manager.cc:3040: 79 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: c50d046abda4a2ab6ab4bafb37b97be86733da4e69c0b14606e92e5c, IP: 172.21.0.25) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.21.0.25`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2023-04-08 01:23:17,586 E 17548 17548] (raylet) node_manager.cc:3040: 81 Workers (tasks / actors) killed due

In [19]:
#Save latest ckpt point
cwd_checkpoint = f"results/org_{agent_name}_{date}_{ep}"
trainer.save(cwd_checkpoint)
#Extract model weights 
model_weights = trainer.get_policy().get_weights()
print('passed model weights')
config2 = TD3Config()
print('config created')
config2 = config2.environment(env_config={'hmax':500,'initial_amount':1000000})  
config2 = config2.rollouts(num_rollout_workers=0) 
config2 = config2.framework(framework="torch")
config2["model"]["fcnet_hiddens"] = [256, 256, 256]
trainer2 = config2.build(env="finrl") 
trainer2.get_policy().set_weights(model_weights)
print('New Weights loaded. ')
ckpt2 = f"{cwd_checkpoint}_wt"
trainer2.save(ckpt2)
zipped_filename = f'ckpt_wt{date}_{ep}.zip'


passed model weights
config created


/home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")


New Weights loaded. 
zip_file created.


FileNotFoundError: [Errno 2] No such file or directory: 'file zipped at ckpt_wt230407_201.zip'

(raylet) [2023-04-08 03:49:38,011 E 17548 17548] (raylet) node_manager.cc:3040: 75 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: c50d046abda4a2ab6ab4bafb37b97be86733da4e69c0b14606e92e5c, IP: 172.21.0.25) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.21.0.25`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


In [5]:
zipped = zipfilem(ckpt2,zipped_filename)
st = sendfile('ckpt_wt230407_201.zip')
print(f'file {zipped} from {ckpt2} ; {st}')

AttributeError: 'NoneType' object has no attribute 'sendall'